In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Data processing notebook
Here are the files for stock and news. Some basic processing were done to make it more concise

In [ ]:
# df_stock = pd.read_csv("SPX.csv")
# df_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [ ]:
# df_stock['Date'] = pd.to_datetime(df_stock['Date'])
# df_stock.set_index('Date', inplace=True)
# df_stock = df_stock.iloc[::-1]
# df_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-04,3406.459961,3486.250000,3405.169922,3443.439941,3443.439941,4783040000
2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000
2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000


In [5]:
df_snp = pd.read_csv("all_stocks_5yr.csv")
df_snp['date'] = pd.to_datetime(df_snp['date'])
df_snp.set_index('date', inplace=True)
df_snp = df_snp.iloc[::-1]
df_snp.head()

,open,high,low,close,volume,Name
date,,,,,,
2018-02-07,72.70,75.00,72.69,73.86,4534912,ZTS
2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS
2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS


In [6]:
snp500_tickers = df_snp['Name'].unique()
snp500_tickers = snp500_tickers.tolist()

In [8]:
df_news = pd.read_csv("analyst_ratings_processed.csv")
# df_news.head()

In [9]:
df_news = df_news.dropna(subset=['title'])  # doesn't seem necessary
del df_news['Unnamed: 0']  # this was the index
df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce', utc=True)
df_news = df_news.dropna(subset=['date'])
df_news.set_index('date', inplace=True)
df_news.head()

,title,stock
date,,
2020-06-05 14:30:00+00:00,Stocks That Hit 52-Week Highs On Friday,A
2020-06-03 14:45:00+00:00,Stocks That Hit 52-Week Highs On Wednesday,A
2020-05-26 08:30:00+00:00,71 Biggest Movers From Friday,A
2020-05-22 16:45:00+00:00,46 Stocks Moving In Friday's Mid-Day Session,A
2020-05-22 15:38:00+00:00,B of A Securities Maintains Neutral on Agilent...,A


In [10]:
df_news = df_news[df_news['stock'].isin(snp500_tickers)]


In [11]:
df_news.shape, df_snp.shape

((383051, 2), (619040, 6))

In [15]:
df_snp.tail()


,open,high,low,close,volume,Name
date,,,,,,
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL


In [ ]:
# df_stock = df_stock[df_stock.index >= '2010-01-01']
# df_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-04,3406.459961,3486.250000,3405.169922,3443.439941,3443.439941,4783040000
2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000
2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000
